In [1]:
from utils import *

# test data generation

In [2]:
rnn_size = 1

data_size = 9
num_data = 3

x_data = []
y_data = []

normalizer = 1

for i in range(num_data):
    input_temp = []
    for j in range(i,i+data_size):
        input_temp.append(j*normalizer)
    x_data.append(input_temp)
    output_temp = normalizer*(i+data_size)
    y_data.append(output_temp) 

x_data = np.array(x_data, dtype = np.float32)
x_data = np.reshape(x_data, [num_data,data_size,1])
y_data = np.array(y_data, dtype = np.float32)
y_data = np.reshape(y_data,[-1,1])

print(x_data.shape)
print(y_data.shape)

(3, 9, 1)
(3, 1)


# Graph restore

In [3]:
sess = tf.Session()
saver = tf.train.import_meta_graph('./save/rnn.meta')
saver.restore(sess, './save/rnn')
print_keys("trainable_variables")

Collection name : trainable_variables
Tensor("Variable/read:0", shape=(1, 1), dtype=float32)
Tensor("Variable_1/read:0", shape=(1,), dtype=float32)
Tensor("rnn/rnn/lstm_cell/weights/read:0", shape=(2, 4), dtype=float32)
Tensor("rnn/rnn/lstm_cell/biases/read:0", shape=(4,), dtype=float32)


# Graph print

In [4]:
print_nodes(tf.get_default_graph())

Graph : <tensorflow.python.framework.ops.Graph object at 0x7f92843dd710>
Placeholder
Placeholder_1
transpose/perm
transpose
Reshape/shape
Reshape
split/split_dim
split
random_normal/shape
random_normal/mean
random_normal/stddev
random_normal/RandomStandardNormal
random_normal/mul
random_normal
Variable
Variable/Assign
Variable/read
random_normal_1/shape
random_normal_1/mean
random_normal_1/stddev
random_normal_1/RandomStandardNormal
random_normal_1/mul
random_normal_1
Variable_1
Variable_1/Assign
Variable_1/read
rnn/rnn/Shape
rnn/rnn/strided_slice/stack
rnn/rnn/strided_slice/stack_1
rnn/rnn/strided_slice/stack_2
rnn/rnn/strided_slice
rnn/rnn/stack/1
rnn/rnn/stack
rnn/rnn/zeros/Const
rnn/rnn/zeros
rnn/rnn/stack_1/1
rnn/rnn/stack_1
rnn/rnn/zeros_1/Const
rnn/rnn/zeros_1
rnn/rnn/lstm_cell/weights/Initializer/random_uniform/shape
rnn/rnn/lstm_cell/weights/Initializer/random_uniform/min
rnn/rnn/lstm_cell/weights/Initializer/random_uniform/max
rnn/rnn/lstm_cell/weights/Initializer/random_unif

# Example

In [5]:
temp = tf.get_collection("trainable_variables")[0]
print(temp.get_shape().as_list())
print(temp.op.name)
print(sess.run(temp))

[1, 1]
Variable
[[ 8.80031204]]


# Create var_dict

In [6]:
var_dict = get_var_dict(sess, "trainable_variables")

In [7]:
pprint.pprint(var_dict)

{'Variable': {'shape': [1, 1], 'value': array([[ 8.80031204]], dtype=float32)},
 'Variable_1': {'shape': [1], 'value': array([ 6.62012672], dtype=float32)},
 'rnn/rnn/lstm_cell/biases': {'shape': [4],
                              'value': array([ 0.82305032,  1.11515999,  0.60988927, -2.59389567], dtype=float32)},
 'rnn/rnn/lstm_cell/weights': {'shape': [2, 4],
                               'value': array([[-0.07742871,  0.6309644 ,  1.12232792,  0.46602029],
       [ 1.38758624,  2.42759705,  0.84666395, -2.32156873]], dtype=float32)}}


# Reset graph

In [8]:
sess.close()
tf.reset_default_graph()
print_nodes(tf.get_default_graph())
print_keys("trainable_variables")

Graph : <tensorflow.python.framework.ops.Graph object at 0x7f92823c6278>
Collection name : trainable_variables


# Basic variable restore

In [9]:
weight = restore_with_var_dict("Variable", var_dict)
bias = restore_with_var_dict("Variable_1", var_dict)

# LSTM variable restore

In [10]:
train_x = tf.placeholder('float', [None, data_size,1 ])
train_y = tf.placeholder('float', [None,1])

train_x_temp = tf.transpose(train_x, [1,0,2])
train_x_temp = tf.reshape(train_x_temp, [-1,1])
train_x_temp =tf.split(train_x_temp, num_or_size_splits=int(data_size), axis=0)

with tf.variable_scope("restore") as scope:
    lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size) # varaible is not created here
    
    # Declare same variable with get_variable, name is very important
    tf.get_variable(name = 'rnn/lstm_cell/weights'
                    , shape = var_dict['rnn/rnn/lstm_cell/weights']['shape']
                    , initializer = tf.constant_initializer(var_dict['rnn/rnn/lstm_cell/weights']['value']))
    tf.get_variable(name = 'rnn/lstm_cell/biases'
                    , shape = var_dict['rnn/rnn/lstm_cell/biases']['shape']
                    , initializer = tf.constant_initializer(var_dict['rnn/rnn/lstm_cell/biases']['value']))
    
    # get_variable in here gets the variable declared before 
    scope.reuse_variables()
    train_outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, train_x_temp, dtype = tf.float32)
    train_output = tf.matmul(train_outputs[-1], weight) + bias

In [11]:
print_keys("trainable_variables")

Collection name : trainable_variables
Tensor("Variable/read:0", shape=(1, 1), dtype=float32)
Tensor("Variable_1/read:0", shape=(1,), dtype=float32)
Tensor("restore/rnn/lstm_cell/weights/read:0", shape=(2, 4), dtype=float32)
Tensor("restore/rnn/lstm_cell/biases/read:0", shape=(4,), dtype=float32)


# Initialize uninitailized variable
* In this step, the variables in lstm modules are restored using get_variable()

In [12]:
sess = tf.Session()
sess.run(master_initializer(sess))

In [13]:
print(sess.run(train_output, feed_dict = {train_x : x_data}))
print(y_data)

[[ 11.22189713]
 [ 11.8597517 ]
 [ 12.46911812]]
[[  9.]
 [ 10.]
 [ 11.]]
